In [1]:
using Pkg
Pkg.activate("../.")

In [2]:
using Tullio, HDF5

In [3]:
using Revise, DeconvOptim, Napari, Images, FourierTools, Napari, FFTW, TestImages, Noise, CUDA, Zygote, LinearAlgebra
FFTW.set_num_threads(12)

┌ Info: napari version
│   version = 0.4.10
└ @ Napari /home/fxw/.julia/packages/Napari/6LgKu/src/Napari.jl:48
┌ Info: /home/fxw/julia/PythonEnvs/python_napari/lib/python3.8/site-packages/napari
└ @ Napari /home/fxw/.julia/packages/Napari/6LgKu/src/Napari.jl:49
┌ Info: DeconvOptim.jl: CUDA.jl is loaded, so include GPU functionality
└ @ DeconvOptim /home/fxw/julia/DeconvOptim.jl/src/requires.jl:6


In [4]:
CUDA.allowscalar(false)

In [5]:
img = permutedims(h5read("../../../../Nextcloud/Uni/DeconvOptim.jl/good_data/data.h5", "data"), (2,1,3))
psf = ifftshift(h5read("../../../../Nextcloud/Uni/DeconvOptim.jl/good_data/psf.h5", "psf"));

In [6]:
# download here http://bigwww.epfl.ch/deconvolution/data/microtubules/
#img = Float32.(load("data.tif"));
#psf = ifftshift(Float32.(load("psf.tif")));

img_c = CuArray(img)
psf_c = CuArray(psf);

In [23]:
reg = TV(num_dims=3)

#156 (generic function with 1 method)

In [16]:
reg_c = TV_cuda(num_dims=3)

#25 (generic function with 1 method)

In [8]:
loss = Poisson()

poisson_aux (generic function with 2 methods)

In [33]:
@time res_lr = richardson_lucy_iterative(1f-5 .+ img, psf, iterations=300, λ=0.002f0, regularizer=nothing);

 40.091097 seconds (276.15 k allocations: 531.301 MiB, 0.19% gc time, 0.07% compilation time)


In [32]:
h5write("/home/fxw/Nextcloud/Uni/DeconvOptim.jl/good_data/DeconvOptim_lucy_richardson.h5", "res", res_lr)

In [30]:
GC.gc(true)
CUDA.@time res_lr_c = richardson_lucy_iterative(1f-5 .+ img_c, psf_c, iterations=300, λ=0.002f0, regularizer=nothing);

  3.053164 seconds (33.02 k CPU allocations: 1.479 MiB) (1.22 k GPU allocations: 76.297 GiB, 0.07% gc time of which 80.25% spent allocating)


In [27]:
GC.gc(true)
@time res_TV, o = deconvolution(img, psf, iterations=45, regularizer=reg, λ=0.001f0);

 39.858604 seconds (91.13 k allocations: 25.216 GiB, 3.06% gc time)


In [28]:
GC.gc(true)
@time res_no_TV, o = deconvolution(img, psf, iterations=45, regularizer=nothing, λ=0.001f0);

 32.893269 seconds (69.36 k allocations: 19.465 GiB, 2.98% gc time)


In [29]:
h5write("/home/fxw/Nextcloud/Uni/DeconvOptim.jl/good_data/DeconvOptim_lucy_with_TV.h5", "res", res_TV)
h5write("/home/fxw/Nextcloud/Uni/DeconvOptim.jl/good_data/DeconvOptim_lucy_no_TV.h5", "res", res_no_TV)

In [19]:
GC.gc(true)
CUDA.@time res_c, o = deconvolution(img_c, psf_c, iterations=45, regularizer=reg_c, λ=0.001f0);

  4.459219 seconds (4.07 M CPU allocations: 72.670 MiB, 3.43% gc time) (3.75 k GPU allocations: 137.183 GiB, 40.07% gc time of which 90.99% spent allocating)


In [17]:
GC.gc(true)
CUDA.@time res_c, o = deconvolution(img_c, psf_c, iterations=45, regularizer=nothing, λ=0.001f0);

  2.198520 seconds (3.07 M CPU allocations: 51.350 MiB, 1.13% gc time) (1.86 k GPU allocations: 34.782 GiB, 9.04% gc time of which 85.36% spent allocating)
